In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [6]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding="utf8")

In [7]:
from langchain.indexes import VectorstoreIndexCreator

In [8]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [9]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [10]:
response = index.query(query)

In [11]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, 100% polyester fabric, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated sun protection, 52% polyester and 48% nylon fabric, wrinkle-free, quickly evaporates perspiration, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated sun protection, 78% nylon and 22% Lycra Xtra Life fiber fabric, wicks moisture for quick-drying comfort, abrasion resistant, fits comfortably over swimsuit |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated sun protection, 71% Nylon and 29% Polyester fabric, wrinkle resistant, front and back cape venting, two front bellows pockets |

All four shirts provide UPF 50+ rated sun protection, blocking 98% of the sun's harmful rays. The fabrics vary from 100% polyester, 52% polyester and 48

Embeddings create numeric representations of text, with similar numeric scores representing similar sentences/pieces of text. LLMs are used to create embeddings

Vector databases hold chunks of text, encoding them with the method above. We can then pull out the most relevant chunks of text for a query and include those when sending a prompt to the LLM

In [14]:
loader = CSVLoader(file_path=file, encoding='UTF-8')

In [15]:
docs = loader.load()

In [16]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [17]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [18]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [19]:
print(len(embed))

1536


In [20]:
print(embed[:5])

[-0.02221718244254589, 0.0065193031914532185, -0.01825656369328499, -0.03920383378863335, -0.01435881294310093]


In [21]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [22]:
query = "Please suggest a shirt with sunblocking"

In [23]:
docs = db.similarity_search(query)

In [24]:
len(docs)

4

In [25]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [26]:
retriever = db.as_retriever()

In [27]:
llm = ChatOpenAI(temperature = 0.0)


In [28]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [29]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [30]:
display(Markdown(response))

| Name | Description | 
| --- | --- | 
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Recommended by The Skin Cancer Foundation. | 
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front and back cape venting. Made with 52% polyester and 48% nylon. | 
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and front and back cape venting. Made with 71% nylon and 29% polyester. | 
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, traditional fit, and front and back cape venting. Made with 100% polyester. | 

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are made with high-performance fabrics that are moisture-wicking, wrinkle-resistant, and abrasion-resistant. The Men's Plaid Tropic Shirt and Men's Tropical Plaid Short-Sleeve Shirt both feature front and back cape venting for added breathability. The Sun Shield Shirt is recommended by The Skin Cancer Foundation as an effective UV protectant.

In [31]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", # 'stuffs' all the docs into context
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [32]:
response = qa_stuff.run(query)



> Entering new  chain...

> Finished chain.


In [33]:
display(Markdown(response))

Here are some options for shirts with sun-blocking:

1. Sun Shield Shirt: This high-performance sun shirt is guaranteed to protect from harmful UV rays. It is made of 78% nylon and 22% Lycra Xtra Life fiber and is UPF 50+ rated.

2. Men's Plaid Tropic Shirt, Short-Sleeve: This shirt is designed for fishing and offers UPF 50+ coverage. It is made of 52% polyester and 48% nylon and is machine washable and dryable.

3. Men's TropicVibe Shirt, Short-Sleeve: This shirt has built-in UPF 50+ and is made of 71% Nylon and 29% Polyester. It is wrinkle-resistant and has front and back cape venting.

4. Men's Tropical Plaid Short-Sleeve Shirt: This shirt is rated UPF 50+ for superior protection from the sun's UV rays. It is made of 100% polyester and is wrinkle-resistant.

In [34]:
response = index.query(query, llm=llm)

In [35]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Chain types:

Stuff - stuffs the relevant documents into context

MapReduce - aggregates chunks using LLM

Refine - iteratively aggregates chunks, building on context with each iteration, instead of building separate context for each chunk

MapRerank - ranks chunks and includes the most relevant one